In [1]:
import copy

In [2]:
from tqdm import tqdm

In [3]:
import wordfreq
from wordfreq import word_frequency, zipf_frequency

In [ ]:
import torch
import torch.nn as nn

In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForMaskedLM

In [ ]:
processed_data = [{'sentid': 3145019, 
                   'ylabels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'S', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'E', 'O', 'B', 'I', 'I', 'E', 'O', 'O', 'B', 'I', 'E', 'B', 'I', 'I', 'I', 'E', 'O', 'B', 'I', 'I', 'I', 'E', 'O', 'O', 'S', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 
                   'yids': [4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 0, 2, 4, 0, 1, 1, 2, 4, 4, 0, 1, 2, 0, 1, 1, 1, 2, 4, 0, 1, 1, 1, 2, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4], 'text': 'The left lower extremity runoff reveals mild to moderate disease in the SFA and 2-vessel runoff with an occluded posterior tibial artery that reconstitutes at the level of the ankle . 4 .', 
                   'words': ['The', 'left', 'lower', 'extremity', 'runoff', 'reveals', 'mild', 'to', 'moderate', 'disease', 'in', 'the', 'SFA', 'and', '2-vessel', 'runoff', 'with', 'an', 'occluded', 'posterior', 'tibial', 'artery', 'that', 'reconstitutes', 'at', 'the', 'level', 'of', 'the', 'ankle', '.', '4', '.'], 
                   'tokens': ['[CLS]', 'The', 'left', 'lower', 'ex', '##tre', '##mity', 'runoff', 'reveals', 'mild', 'to', 'moderate', 'disease', 'in', 'the', 'SF', '##A', 'and', '2', '-', 'vessel', 'runoff', 'with', 'an', 'o', '##cc', '##luded', 'posterior', 't', '##ibi', '##al', 'artery', 'that', 're', '##con', '##st', '##it', '##utes', 'at', 'the', 'level', 'of', 'the', 'ankle', '.', '4', '.', '[SEP]'], 
                   'token_ids': [101, 1109, 1286, 2211, 4252, 7877, 15455, 21814, 7189, 10496, 1106, 8828, 3653, 1107, 1103, 18659, 1592, 1105, 123, 118, 5832, 21814, 1114, 1126, 184, 19515, 27567, 16530, 189, 21883, 1348, 18593, 1115, 1231, 7235, 2050, 2875, 20311, 1120, 1103, 1634, 1104, 1103, 10845, 119, 125, 119, 102], 
                   'entities': [{'uid': 11086, 'start_token': 40, 'end_token': 41, 'term': 'vertebral level', 'term_frequency': 2.81}, {'uid': 11087, 'start_token': 24, 'end_token': 27, 'term': 'occlude', 'term_frequency': 1.8}, {'uid': 11088, 'start_token': 7, 'end_token': 8, 'term': 'runoff', 'term_frequency': 3.53}, {'uid': 11089, 'start_token': 27, 'end_token': 32, 'term': 'posterior tibial artery', 'term_frequency': 2.35}, {'uid': 11090, 'start_token': 33, 'end_token': 38, 'term': 'reconstitutes', 'term_frequency': 1.36}, {'uid': 11092, 'start_token': 18, 'end_token': 22, 'term': '2-vessel runoff', 'term_frequency': 3.46}, {'uid': 11093, 'start_token': 15, 'end_token': 17, 'term': 'superficial femoral artery', 'term_frequency': 2.75}], 
                   'word_index2token_indices': {0: [1], 1: [2], 2: [3], 3: [4, 5, 6], 4: [7], 5: [8], 6: [9], 7: [10], 8: [11], 9: [12], 10: [13], 11: [14], 12: [15, 16], 13: [17], 14: [18, 19, 20], 15: [21], 16: [22], 17: [23], 18: [24, 25, 26], 19: [27], 20: [28, 29, 30], 21: [31], 22: [32], 23: [33, 34, 35, 36, 37], 24: [38], 25: [39], 26: [40], 27: [41], 28: [42], 29: [43], 30: [44], 31: [45], 32: [46]}}]

In [ ]:
class TermWeighting(object):
    def __init__(self):
        self.weight_calculator
        
    def get_weight(self, term):
        pass
        
    
    

In [ ]:
class TermFrequency(object):
    def __init__(self):
        pass
    
    def get_weight(self, term, mode='zipf', lang = 'en'):
        # word freq = word probability
        if mode == 'word':
            term_weight = word_frequency(term, lang)
        
        # Zipf freq = log10(bilion * word_frequency)
        elif mode == 'zipf':
            term_weight = zipf_frequency(term, lang)
        
        else:
            term_weight = None
        
        return term_weight
        
    def get_weights(self, processed_data, target_entities = 'entities'):
        temp_processed_data = []
        for processed_sentence in tqdm(processed_data):
            token_ids = processed_sentence['token_ids']; 
            entities = processed_sentence[target_entities]
            
            temp_entities = []
            for entity in entities:
                term = entity['term']
                entity['term_frequency'] = self.get_weight(term, mode = 'zipf', lang = 'en')
                temp_entities.append(entity)
            processed_sentence[target_entities] = temp_entities
            temp_processed_data.append(processed_sentence)
        return temp_processed_data

In [ ]:
"""
Class for get BERT-based term weighting
__init__(model_name = ""): 1) initialize tokenizer and model, 2) set [MASK]ing func


"""

class MLM_weight(object):
    def __init__(self, model_name, mode='mask'):
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        
        pretrained_model_config = AutoConfig.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.mask_id = self.tokenizer.mask_token_id
        
        
        # self.preprocessor = self.set_preprocessor()
        
        self.model = AutoModelForMaskedLM.from_pretrained(model_name)
        self.model.to(self.device)
        self.model.eval()
        
        # set mode
        # mode == (mask, flat, pll)
        self.mode = mode
        
    def mask_sentence_generator(self,token_ids, entities):
        mask_id = self.mask_id
        
        masked_token_sentences = []
        for entity in entities:
            tokens = copy.copy(token_ids)
            start_token = entity['start_token']; end_token = entity['end_token']; uid = entity['uid']
            if self.mode == 'mask':
                masked_sentences = tokens[:start_token]  + [mask_id] * (end_token - start_token) + tokens[end_token:]
            else:
                masked_sentences = tokens
            # entity['masked_sentences'] = masked_sentences
            masked_token_sentences.append(masked_sentences)
            
        
        return masked_token_sentences    
        
    def get_masked_weights(self, masked_token_sentences, entities, golden_tokens):
        def make_inputs(masked_token_sentences, golden_tokens):
            device = self.device
            # print(masked_token_sentences)
            token_len = len(masked_token_sentences[0])
            
            input_ids = [masked_token_sentence for masked_token_sentence in masked_token_sentences]
            attention_mask = [[1]*token_len for masked_token_sentence in masked_token_sentences] 
            token_type_ids = [[0]*token_len for masked_token_sentence in masked_token_sentences] 
            labels = [golden_tokens for masked_token_sentence in masked_token_sentences]
            
            
            input_ids = torch.tensor(input_ids).to(device)
            attention_mask = torch.tensor(attention_mask).to(device)
            token_type_ids = torch.tensor(token_type_ids).to(device)
            labels = torch.tensor(labels).to(device)
            
            return input_ids, attention_mask, token_type_ids, labels
        
        def nll(masked_logits, golden, spans):
            start_token, end_token = spans
            # masked_logits.cpu().data; golden.cpu().data; 
            # masked_logits = masked_logits[start_token:end_token]
            # golden = golden[start_token:end_token]
            softmax = torch.nn.Softmax(dim=-1)

            
            masked_probs = softmax(masked_logits)
            
           
            golden = [[gold] for gold in golden]
            golden = torch.tensor(golden)
            # print(golden)
            masked_answer_probs = torch.gather(masked_probs, 1, golden)[start_token: end_token]
            
            
            # print(self.tokenizer.decode(torch.squeeze(golden[start_token: end_token], -1)))
            # print(torch.log(masked_answer_probs))
            # print()
            
            masked_negative_log_likelihood = -1 * torch.mean(torch.log(masked_answer_probs))
            # masked_negative_log_likelihood.item() 
            
            return masked_negative_log_likelihood.item()
            
        model = self.model
        
        input_ids, attention_mask, token_type_ids, labels = make_inputs(masked_token_sentences, golden_tokens)
        
        outputs = model(input_ids, attention_mask=attention_mask,labels=labels, token_type_ids = token_type_ids)
        
        loss = outputs[0]; logits = outputs[1].cpu().data;
        
        weights = []
        # print(logits)
        for masked_token_sentence, masked_logits in zip(entities,logits):
            start_token = masked_token_sentence['start_token']; end_token = masked_token_sentence['end_token']
            masked_nll = nll(masked_logits, golden_tokens, (start_token, end_token))
            weight = masked_nll
            weights.append(weight)
        del(input_ids); del(attention_mask); del(token_type_ids); del(labels); del(logits)
        # print(outputs)
        # print(outputs[1].shape)
        # print(self.tokenizer.decode(golden_tokens))
        # print(self.tokenizer.decode(input_ids[1]))
        
        return weights
    
    def get_weights(self, processed_data, target_entities = 'UMLS_entities'):
        temp_processed_data = []
        for processed_sentence in tqdm(processed_data):
            
            token_ids = processed_sentence['token_ids']; 
            entities = processed_sentence[target_entities];
            if entities:
                # temp_entities = [copy.copy(entity )for entity in entities]
                masked_token_sentences = self.mask_sentence_generator(token_ids, entities)

                weights = self.get_masked_weights(masked_token_sentences, entities, token_ids)

                temp_entities = []
                for weight, entity in zip(weights, entities):
                    entity['MLM_weight'] = weight
                    temp_entities.append(entity)
                processed_sentence[target_entities] = temp_entities
                temp_processed_data.append(processed_sentence)
            else: 
                temp_processed_data.append(processed_sentence)
        return temp_processed_data

In [ ]:
TF_weighting = TermFrequency()
MLM_weighting.get_weights()

In [ ]:
MLM_weighting = MLM_weight("bert-base-cased", mode='mask')
MLM_weighting.get_weights(processed_data, target_entities='entities')[0]['entities']

In [6]:
word_frequency('diuretics', lang='en')

2.75e-07